In [ ]:
import missingno as msno
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import statsmodels.api as sm
import yaml
from scipy import stats
from scipy.stats import normaltest
from statsmodels.graphics.gofplots import qqplot
from sqlalchemy import create_engine
from sqlalchemy import inspect
from matplotlib import pyplot as plt

# Accessing the df

In [ ]:
def credentials_reader():
    with open('credentials.yaml', 'r') as file:
        credentials = yaml.safe_load(file)
        return credentials

In [ ]:
class RDSDatabaseConnector:
    def __init__(self, credentials):
        self.credentials = credentials
        
    def initiate_engine(self):

        self.DATABASE_TYPE = 'postgresql'
        self.DBAPI = 'psycopg2'
        self.HOST = 'eda-projects.cq2e8zno855e.eu-west-1.rds.amazonaws.com'
        self.USER = 'loansanalyst'
        self.PASSWORD = 'EDAloananalyst'
        self.DATABASE = 'payments'
        self.PORT = 5432

        self.engine = create_engine(f"{self.DATABASE_TYPE}+{self.DBAPI}://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}")
        self.engine.execution_options(isolation_level='AUTOCOMMIT').connect()
        
        inspector = inspect(self.engine)
        self.table_inspector = inspector.get_table_names()
        print(self.table_inspector)

    def database_to_dataframe(self):
        ######### maybe add wiith self.engine.connect() as connection: (only if it doesn't work like this)
        self.loans = pd.read_sql_table('loan_payments', self.engine)
        return self.loans.head(10), self.loans.tail(10)
        
    # Saves the data to your current pathway as eda.csv
    def saves_data_locally(self):
        self.loans_df.to_csv('eda.csv', sep=',', index=False, encoding='utf-8')

    def load_localdata_to_dataframe(self):
        # Defining the columns to read
        usecols = ["id", "member_id","loan_amount", "funded_amount", "funded_amount_inv", "term", "int_rate", "instalment", "grade", "sub_grade", "employment_length", "home_ownership", "annual_inc", "verification_status", "issue_date", "loan_status", "payment_plan", "purpose", "dti", "delinq_2yrs", "earliest_credit_line", "inq_last_6mths", "mths_since_last_record", "open_accounts", "total_accounts", "out_prncp", "out_prncp_inv", "total_payment", "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_payment_date", "last_payment_amount", "next_payment_date", "last_credit_pull_date", "collections_12_mths_ex_med", "mths_since_last_major_derog", "policy_code", "application_type"]
    # Read data with subset of columns
        loan_data_df = pd.read_csv("/Users/joeybest/Ai Core/EDA/exploratory-data-analysis---customer-loans-in-finance334/eda.csv", index_col="id", usecols=usecols)
        return loan_data_df

In [ ]:
# reads credentials from yaml file
credentials = credentials_reader()

RDSDatabaseConnector(credentials)
# next creates instance of the rds connector
loan_data = RDSDatabaseConnector(credentials)
loan_data.initiate_engine()
loan_data.database_to_dataframe()

extracted_data_frame = loan_data.database_to_dataframe()
# print(extracted_data_frame)

# was after extracted dataframe variable but it seems to work without it:
# pd.DataFrame

loan_data.save_to_csv("eda.csv")
# saves CSV file

table_of_loans = loan_data.load_localdata_to_dataframe()
print(table_of_loans)

In [ ]:
table_of_loans = loan_data.load_localdata_to_dataframe()

In [ ]:
print(table_of_loans.head(10))
print(table_of_loans.tail(10))

In [ ]:
print(table_of_loans.info())

In [ ]:
print(table_of_loans.describe())

In [ ]:
print(f"std loan_amount = {table_of_loans['loan_amount'].std()}, mean loan_amount = {table_of_loans['loan_amount'].mean()}")
print(f"Max loan: {table_of_loans['loan_amount'].max()}, Min loan:{table_of_loans['loan_amount'].min()}")

# Exploring the data

In [ ]:
class DataFrameInfo:

    def __init__(self, df_info):
        self.df_info = df_info

    def check_column_datatypes(self): 
        return self.df_info.dtypes

    def extract_statistical_values(self, columns):
        for col in columns:
            print(f'Statistics for Col: {col}')
            return self.df_info[col].describe()

    def count_unique_categories(self, columns):
        self.columns = self.df_info.select_dtypes(include=['category']).columns
        df_unique = self.df_info[columns[:]]
        return df_unique.nunique()
        # orrrrrrrr
    #def count_unique_categories(self, df_info: pd.DataFrame, column_name: str):
     #   return len(df_info[column_name].unique())

    def shape_of_dataframe(self):
        print(f'Shape of DataFrame: [{self.df_info.shape[0]} rows x {self.df_info.shape[1]} columns]\n')

    def num_of_nulls(self):
        cols = self.df_info.columns
        # Define an empty list of null column rows
        null_coulmn_rows = []
        # Populate list of null column rows
        for col in cols:
            # if self.df[col].isnull().sum() > 0:
            null_coulmn_rows.append([col, self.df_info[col].count(), 100*(self.df_info[col].isnull().sum()/len(self.df_info))])
        
        # Convert the list into dataframe rows
        data = pd.DataFrame(null_coulmn_rows)
        # Add columns headers
        data.columns = ['column', 'count', '% null count']  
        return data

In [ ]:
df_df = DataFrameInfo(table_of_loans)


In [ ]:
print(df_df.check_column_datatypes())

In [ ]:
print(f"The no. of Unique Items in {df_df.count_unique_categories(['grade'])}")
print('')
print(df_df.extract_statistical_values(['loan_amount']))
print(' ')
print(df_df.shape_of_dataframe())

In [ ]:
print(df_df.num_of_nulls())

# Transforming pt 1

In [ ]:
class DataTransform:

    def __init__(self, df_info):
        self.df_info = df_info

    def to_interger(self, column):
        for col in column:
            self.df_info[col] = self.df_info[col].fillna(0).astype('int32')
            # converts to int and fills NULL with 0

    def to_boolean(self, column_name):
        mask = {'n': False, 'y': True}
        self.df_info[column_name].map(mask)
        self.df_info[column_name] = self.df_info[column_name].astype('bool')
        print(self.df_info[column_name].unique())

    def to_object(self, column):
        for col in column:
            self.df_info[col] = self.df_info[col].astype(object)
    
    def to_float(self, column):
        for col in column:
            self.df_info[col] = self.df_info[col].astype('float64')

    def to_rounded_float(self, column, decimal_places):
        self.df_info[column] = self.df_info[column].apply(lambda x: round(x, decimal_places))

    def to_category(self, column):
        for col in column:
            self.df_info[col] = self.df_info[col].astype('category')

    def to_numerical_column(self, column):
        for col in column:
            pd.to_numeric(self.df_info[col])

    def extract_integer_from_string(self, column):
        for col in column:
            self.df_info[col] = self.df_info[col].str.extract('(\d+)')

    def strings_to_dates(self, column):
        for col in column:
            self.df_info[col] = pd.to_datetime(self.df_info[col], errors='coerce', format="%b-%Y")#.dt.to_period('M') - method converts the datetime to a period (M) which is a date that contains only the month and year since this is the resolution of the data provided.

    def replace_string_text(self, column_name, original_string: str, new_string: str):
        self.df_info[column_name].replace(original_string, new_string)

    def rename(self, column_name, new_column_name):
        self.df_info.rename(columns={col_name: new_col_name})

    def drop_column(self, column):
        for col in column:
            self.df_info.drop(col, axis=1, inplace=True)

    def remove_null_rows(self, column_name):
        '''
        This method is used to remove rows within the dataframe where data points from a specified column are null.
        '''
        self.df_info.dropna(subset=column_name, inplace=True)

    def save_transformed_data(self, filename='full_loan_data.csv'):
        self.df_info.to_csv(filename, index=False)

In [ ]:
if __name__ ==  "__main__":
    table_of_loans = pd.read_csv('eda.csv')
    Transform = DataTransform(table_of_loans)

    Transform.to_boolean('payment_plan')
    # transforms n to false and y to true in the payment plan column

    to_object_columns = ['id', 'member_id', 'policy_code']
    Transform.to_object(to_object_columns)
    
    convert_categories = ['grade', 'sub_grade', 'home_ownership', 'verification_status', 'loan_status', 'purpose', 'employment_length']
    Transform.to_category(convert_categories)
    # transforms column values to catagories

    string_month_and_year = ['last_credit_pull_date', 'next_payment_date', 'last_payment_date', 'earliest_credit_line', 'issue_date']
    Transform.strings_to_dates(string_month_and_year)
    # transforms strings to a datetime format MonthYear

    string_numbers = ['term']
    Transform.extract_integer_from_string(string_numbers)

    numericals = ['term', 'mths_since_last_record', 'mths_since_last_major_derog', 'mths_since_last_delinq', 'mths_since_last_record']
    Transform.to_numerical_column(numericals)

    drop_cols = ['funded_amount', 'application_type', 'policy_code', 'out_prncp_inv', 'total_payment_inv']
    Transform.drop_column(drop_cols)
    # funded_amount is missing some data but contains the same data as funded_amount_inv, so we can drop it
    # out_prncp_inv contains the same data as out_prncp, so we can drop it
    # total_payment_inv contains the same data as total_payment, so we can drop it
    # application_type and policy_code are the same for everyone and doesn't provide us with much info
    
    int_numbers = ['loan_amount', 'funded_amount_inv', 'annual_inc', 'term', 'open_accounts', 'total_accounts', 'collections_12_mths_ex_med', 'mths_since_last_delinq', 'mths_since_last_major_derog']
    Transform.to_interger(int_numbers)
    
    Transform.to_rounded_float('collection_recovery_fee', 2)

    # saves a new CSV of the df called 'full_loan_data.csv'
    Transform.save_transformed_data('full_loan_data.csv')

# Transforming pt 2

In [ ]:
class DataFrameTransform:

    def __init__(self, df_transform):
        self.df_transform = df_transform

    def num_of_nulls(self):
        cols = self.df_transform.columns
        # Define an empty list of null column rows
        null_coulmn_rows = []
        # Populate list of null column rows
        for col in cols:
            # if self.df[col].isnull().sum() > 0:
            null_coulmn_rows.append([col, self.df_transform[col].count(), 100*(self.df_transform[col].isnull().sum()/len(self.df_transform))])
        
        # Convert the list into dataframe rows
        data = pd.DataFrame(null_coulmn_rows)
        # Add columns headers
        data.columns = ['column', 'count', '% null count']  
        return data

    def drop_column(self, column):
        '''
        This method removes the listed columns from the dataframe.
        '''
        for col in column:
            self.df_transform.drop(col, axis=1, inplace=True)

    def drop_null_rows(self, column):
        self.df_transform.dropna(subset=column, inplace=True)
    
    def impute_zeros(self, column):
        for col in column:
            self.df_transform[col] = self.df_transform[col].fillna(0)

    def impute_median(self, column):
        for col in column:
            self.df_transform[col] = self.df_transform[col].fillna(self.df_transform[col].median())

    def impute_mean(self, column):
        for col in column:
            self.df_transform[col] = self.df_transform[col].fillna(self.df_transform[col].mean())

    def log_transform(self, column):
        for col in column:
            log_sample = self.df_transform[col] = self.df_transform[col].map(lambda i: np.log(i) if i > 0 else 0)
            t=sns.histplot(log_sample,label="Skewness: %.2f"%(log_sample.skew()) )
            t.legend()
    
    def box_cox_transform(self, column):
        for col in column:
            boxcox_column = self.df_transform[col] + 0.01
            a, b = stats.boxcox(boxcox_column)
            self.df_transform[col] = a 
            t=sns.histplot(boxcox_column,label="Skewness: %.2f"%(boxcox_column.skew()) )
            t.legend()

    def remove_outliers_iqr(self, data, threshold=1.5):
        q1 = np.percentile(data, 25)
        q3 = np.percentile(data, 75)
        iqr = q3 - q1
        lower_bound = q1 - threshold * iqr
        upper_bound = q3 + threshold * iqr
        filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]
        return filtered_data

    def remove_outliers_iqr_dataframe(self, column, threshold=1.5):
        filtered_dataframe = self.df_transform
        for col in column:
            filtered_dataframe[col] = self.remove_outliers_iqr(filtered_dataframe[col], threshold)
        return filtered_dataframe
        
    def save_transformed_data(self, filename='transformed_loan_data.csv'):
        self.df_transform.to_csv(filename, index=False)

    def save_untransformed_data(self, filename='untransformed_loan_data.csv'):
        '''
        This method is used to save the current dataframe as a new CSV file called 'transformed_loan_data.csv'
        '''
        self.df_transform.to_csv(filename, index=False)

In [ ]:
class Plotter:

    def __init__(self, df_transform):
        self.df_transform = df_transform

    def agostino_k2_test(self, col):
        stat, p = normaltest(self.df_transform[col], nan_policy='omit')
        print('Statistics=%.3f, p=%.3f' % (stat, p))

    def histogram(self, col):
        self.df_transform[col].hist(bins=40)

    def density_plot(self, col):
        sns.histplot(data=self.df_transform[col], kde=True)
        sns.despine()

    def boxplot(self, col):
        fig = px.box(self.df_transform[col],width=600, height=500)
        fig.show()

    def bar(self, data):
        df = pd.DataFrame(data, index=[0])
        sns.barplot(data=df)

    def scatter(self, col):
        sns.scatterplot(self.df_transform[col])

    def pie(self, col):
        data = self.df_transform[col].value_counts()

        fig = px.pie(values=data.values, names=data.index, title= 'Pie Chart of {self.df_transform[col]}', width=600)
        fig.show()

    def qq_plot(self, col):
        self.df_transform.sort_values(by=col, ascending=True)
        qq_plot = qqplot(self.df_transform[col], scale=1, line='q')
        plt.show()

    def show_missing_nulls(self):
        msno.matrix(self.df_transform)
        plt.show()

    def heatmap(self, col):
        corr = self.df_transform[col].corr()
        mask = np.zeros_like(corr)
        mask[np.triu_indices_from(mask)] = True
        plt.figure(figsize=(10, 8))

        cmap = sns.diverging_palette(220, 10, as_cmap=True)

        sns.heatmap(corr, mask=mask, square=True, linewidths=5,
                    annot=True, cmap=cmap)

        # Show the plot
        plt.show()

    def multi_plot(self, col):
        print(f'The median of {[col]} is {self.df_transform[col].median()}')
        print(f'The mean of {[col]} is {self.df_transform[col].mean()}')
        print(f"Skew of {[col]} column is {self.df_transform[col].skew()}")
        stat, p = normaltest(self.df_transform[col], nan_policy='omit')
        print('Statistics=%.3f, p=%.3f' % (stat, p))
        self.df_transform[col].hist(bins=40)
        sns.histplot(data=self.df_transform[col], kde=True)
        sns.despine()
        self.df_transform.sort_values(by=col, ascending=True)
        qq_plot = qqplot(self.df_transform[col], scale=1, line='q', fit=True)
        plt.show()

    def find_skew(self, col):
        if self.df_transform[col].skew() >= 0.86:
            print('Skewed!')
            print(f' {self.df_transform[col].skew()} is over 0.85')
        else:
            print(f' {self.df_transform[col].skew()} is under 0.85')

    def multi_hist_plot(self, num_cols):
        sns.set(font_scale=0.7)
        f = pd.melt(self.df_transform, value_vars=num_cols)
        g = sns.FacetGrid(f, col="variable", col_wrap=4,
                          sharex=False, sharey=False)
        g = g.map(sns.histplot, "value", kde=True)
        plt.show()

    def multi_qq_plot(self, cols):
        remainder = 1 if len(cols) % 4 != 0 else 0
        rows = int(len(cols) / 4 + remainder)

        fig, axes = plt.subplots(
            ncols=4, nrows=rows, sharex=False, figsize=(12, 6))
        for col, ax in zip(cols, np.ravel(axes)):
            sm.qqplot(self.df_transform[col], line='s', ax=ax, fit=True)
            ax.set_title(f'{col} QQ Plot')
        plt.tight_layout()

    def show_outliers(self):
        #select only the numeric columns in the DataFrame
        df = self.df_transform.select_dtypes(include=['float64'])
        plt.figure(figsize=(18,14))

        for i in list(enumerate(df.columns)):
            fig_cols = 4
            fig_rows = int(len(df.columns)/fig_cols) + 1
            plt.subplot(fig_rows, fig_cols, i[0]+1)
            sns.boxplot(data=df[i[1]]) 

        # Show the plot
        plt.tight_layout()
        plt.show()

    def show_outliers_after_removal(self, dataframe, columns):
        plt.figure(figsize=(18, 14))

        for i, col in enumerate(columns):
            fig_cols = 4
            fig_rows = len(columns) // fig_cols + 1
            plt.subplot(fig_rows, fig_cols, i + 1)
            sns.boxplot(data=dataframe[col])

        plt.tight_layout()
        plt.show()

In [ ]:
table_of_loans = pd.read_csv('full_loan_data.csv')
df_cols = DataFrameTransform(table_of_loans)
plot = Plotter(table_of_loans)

In [ ]:
print(df_cols.num_of_nulls())

In [ ]:
df_cols.impute_zeros(['employment_length'])
# missing values can be assumed to be unemployment
df_cols.impute_median(['int_rate'])
# Missing values imputed to median values 
df_cols.drop_null_rows(['last_payment_date', 'last_credit_pull_date'])
# Data missing too small to impact much, so remove rows
df_cols.drop_column(['mths_since_last_delinq', 'next_payment_date', 'mths_since_last_record', 'mths_since_last_major_derog'])
# Data missing from 57%-88%, so drop column
# df_cols.drop_column(['funded_amount', 'application_type', 'policy_code', 'out_prncp_inv', 'total_payment_inv'])
# columns that shouldve been dropped in DataTransformation stage


In [ ]:
df_cols.save_untransformed_data('untransformed_loan_data.csv')
# to save a version of untransformed data for use in Milestone 4 later on

In [ ]:
# testing int_rate
int_rate_data = table_of_loans['int_rate']

# D’Agostino’s K^2 Test
stat, p = normaltest(int_rate_data, nan_policy='omit')
print('Statistics=%.3f, p=%.3f' % (stat, p))

In [ ]:
plot.show_missing_nulls()

## visualising the data

In [ ]:
plot.multi_plot('total_payment')
# positive skew

In [ ]:
plot.multi_plot('funded_amount_inv')
# normal distribution?

In [ ]:
plot.multi_plot('instalment')
# skewed but not badly, so could be normal

In [ ]:
plot.multi_plot('loan_amount')
# positive skew ?
# normal distribution?, slight positive

In [ ]:
plot.multi_plot('int_rate')
# distribution is normal and  mean and median are prettyy similar, so we'll impute using the median
# normal distribution

In [ ]:
# change around columns as you please

In [ ]:
# skew data
# normal skew?
# Skew of int_rate column is 0.45661978102982004/// logt= -0.38/// boxcox = 0.46
# Skew of total_accounts column is 0.7790400286166349/// logt= -0.68/// boxcox = 0.78
# Skew of loan_amount column is 0.8049220181025606/// logt= -0.68/// boxcox = 0.80
# Skew of funded_amount_inv column is 0.8133743608921553/// logt= -3.33/// boxcox = 0.81
# Skew of dti column is 0.18910057534680505/// logt= -1.98/// boxcox = 0.19

# positive skew:
# Skew of instalment column is 0.9965721690437305/// logt= -0.67/// # ? boxcox = 1
# Skew of open_accounts column is 1.0591835802494733/// logt= -0.47/// # boxcox = 1.06
# Skew of total_rec_prncp column is 1.2626785390807123/// logt= -0.92/// # boxcox = 1.26
# Skew of total_payment column is 1.2698752968451772/// logt= -0.75/// # boxcox = 1.27
# Skew of total_rec_int column is 2.2045848938701638/// logt= -0.56/// # boxcox = 2.20
# Skew of out_prncp column is 2.35405079479137/// logt=0.57/// # boxcox = 2.35
# Skew of last_payment_amount column is 2.4972203995928135/// logt=0.13/// # boxcox = 2.50
# Skew of inq_last_6mths column is 3.253522804381087/// logt=1.97///# boxcox = 3.25
# Skew of delinq_2yrs column is 5.376384516017939 /// # logt=5.42/// boxcox = 5.38
# Skew of annual_inc column is 8.717499091755876/// logt= 0.14/// # boxcox = 8.72
# Skew of total_rec_late_fee column is 13.174553220535644/// logt=5.51///# boxcox = 13.17
# Skew of recoveries column is 14.389298686317593/// logt=3.82/// # boxcox = 14.39
# Skew of collections_12_mths_ex_med column is 20.307695011511576/// # logt=65.06/// boxcox = 20.31
# Skew of collection_recovery_fee column is 27.807756579876514/// logt=5.41/// # boxcox = 27.81

In [ ]:
logt_cols = ['annual_inc', 'total_accounts', 'open_accounts', 'last_payment_amount']
df_cols.log_transform(logt_cols)

In [ ]:
boxcox_cols = ['loan_amount', 'instalment', 'int_rate', 'dti', 'funded_amount_inv', 'total_payment']
df_cols.box_cox_transform(boxcox_cols)

In [ ]:
# code below saves file, but its already saved
df_cols.save_transformed_data('transformed_loan_data.csv')

# Outlier Stage

In [ ]:
transformed_loans = pd.read_csv('transformed_loan_data.csv')
df_cols = DataFrameTransform(transformed_loans)
plot = Plotter(transformed_loans)

In [ ]:
numerical_cols = ['loan_amount','funded_amount_inv', 'int_rate', 'instalment', 'dti', 'annual_inc', 'total_payment', 'total_accounts', 'open_accounts', 'last_payment_amount']
plot.multi_hist_plot(numerical_cols)

In [ ]:
plot.multi_qq_plot(numerical_cols)

In [ ]:
# boxplot of 
plot.show_outliers()

In [ ]:
plot.boxplot(numerical_cols)

In [ ]:
filtered_df = df_cols.remove_outliers_iqr_dataframe(column= numerical_cols, threshold=1.5)
plot.show_outliers_after_removal(dataframe=filtered_df, columns=numerical_cols)

In [ ]:
df_without_outliers = DataFrameTransform(filtered_df)
plot_without_outliers = Plotter(filtered_df)
Transform = DataTransform(filtered_df)

In [ ]:
plot_without_outliers.boxplot(numerical_cols)


In [ ]:
plot_without_outliers.multi_hist_plot(numerical_cols)

In [ ]:
plot_without_outliers.show_missing_nulls()
# removing the outliers has left NAN values, so we will either transform or remove the rows

In [ ]:
df_without_outliers.num_of_nulls()
# values are all very low, impute with the mean!

In [ ]:
# Outliers: If a column has noticeable outliers, the median is more robust and less influenced by extreme values.
to_be_median_imputed = ['loan_amount', 'funded_amount_inv',  'int_rate', 'instalment', 'annual_inc', 'open_accounts', 'total_accounts', 'total_payment', 'last_payment_amount']
df_without_outliers.impute_median(to_be_median_imputed)

In [ ]:
df_without_outliers.num_of_nulls()
plot_without_outliers.show_missing_nulls()

In [ ]:
plot_without_outliers.multi_qq_plot(numerical_cols)

In [ ]:
plot_without_outliers.heatmap(numerical_cols)

In [ ]:
## Threshold for highly correlated columns is 0.85

# Multi-linearity between 'loan_amount', 'instalment' & 'funded_amount_inv'

# funded_amount_inv and loan_amount corrolation = 0.96
# instalmannt and loan_amount corrolation = 0.96
# instalmannt and funded_amount_inv corrolation = 0.93
    # desptie columns being past the threshold, they're all important for the analysis stage, so we wont be dropping any of them!

# total_payment is highly corrolated with loan_amount, funded_amount_inv and instalment too, but only at 0.81, 0.78 and 0.81 respectively
# therefore not passed the 0.85 threshold

In [ ]:
df_cols.save_transformed_data('filtered_loan_data.csv')
# saves data without any outliers

# Milestone 4

## Current state of the Loans

In [ ]:
filtered_df = pd.read_csv('untransformed_loan_data.csv')
# untransformed data, specifically for milestone 4
df_without_outliers = DataFrameTransform(filtered_df)
plot_without_outliers = Plotter(filtered_df)
Transform = DataTransform(filtered_df)

In [ ]:
filtered_df['term'] = filtered_df['term'].replace(0, 36)
# replaces 0's in term with median value(36)

In [ ]:
# loan recovered (total_payment) againts invstor funding (funded_amount_inv) & total amount funded (loan_amount)
total_amount_funded = filtered_df['loan_amount'].sum()
print(f' Total loan amount funded is {total_amount_funded}')
invstor_funding = filtered_df['funded_amount_inv'].sum()
print(f' Total amount invested is {invstor_funding}')
total_payment_sum = filtered_df['total_payment'].sum()
print(f' Total payment recovered is {total_payment_sum}')

In [ ]:
#together and rounded
totals = round(filtered_df[['loan_amount', 'funded_amount_inv', 'total_payment']].sum(), 0)
totals

In [ ]:
# % of loans recovered
print(f'Percentage of the loans recovered against the investor_funding is {(total_payment_sum/invstor_funding)*100}')
print(f'Percentage of the loans recovered against the total_amount_funded is {(total_payment_sum/total_amount_funded)*100}')

per_of__inv_loan_recovered = round((total_payment_sum/invstor_funding)*100, 2)
per_of__total_loan_recovered = round((total_payment_sum/total_amount_funded)*100, 2)
print(f' {per_of__inv_loan_recovered}%')
print(f' {per_of__total_loan_recovered}%')

In [ ]:
data = {
'funded_amount_inv': (filtered_df['total_payment'].sum()/invstor_funding)*100,
'loan_amount': (filtered_df['total_payment'].sum()/total_amount_funded)*100
}

plot_without_outliers.bar(data)

In [ ]:
# Calculate percentage of investor funding recovered
pct_invetor_rec = round(100 * totals.total_payment/totals.funded_amount_inv, 2)
# Calculate percentage of funded amount recovered
pct_total_rec = round(100 * totals.total_payment/totals.loan_amount, 2) 

In [ ]:
data = {'Funding': ['Investor', 'Total'],
            'Percent': [pct_invetor_rec, pct_total_rec]}

# Create a Pandas DataFrame
df = pd.DataFrame(data)
display(df)

df.plot(x="Funding", y="Percent", kind="bar", ylabel="% Recovered") 

In [ ]:
df.plot(y="Percent", kind="pie", ylabel="% Recovered", labels=df['Funding'], startangle=90, xlabel='Funding')
# Add a title and xlabel
plt.title("Recovery Percentage by Funding Source")
plt.legend(title="Funding Source")
# Display the chart
plt.show()

In [ ]:
remaining_amount = round(total_amount_funded - total_payment_sum, 2)
remaining_amount
# whats left to be paid overall

In [ ]:
# Creating a DataFrame with the recovered and remaining amounts
data = {'Amount': [total_payment_sum, remaining_amount]}
df = pd.DataFrame(data, index=['Recovered', 'Remaining'])

# Plotting the pie chart
df.plot(y='Amount', kind='pie', autopct='%1.1f%%', startangle=90)
# Adding a title and legend title
plt.title("Percentage of Amount Recovered")
plt.legend(title="Key")
# Displaying the chart
plt.show()

In [ ]:
# sum of the total_payment and the instalment and multiply this by 6. 
# Then divide this by the sum of the funded amount inv. 
# times By 100 to get this as a percentage

six_month_projection = round((filtered_df["total_payment"].sum()+(filtered_df["instalment"].sum()*6))/(filtered_df["funded_amount_inv"].sum())*100, 2)

In [ ]:
data = {'Funding': ['Total'],
        'Percent': [six_month_projection]}

# Create a Pandas DataFrame
df = pd.DataFrame(data)
display(df)

df.plot(x="Funding", y="Percent", kind="bar", ylabel="% Recovered") 

In [ ]:
# Filter the DataFrame to include only charged off loans
charged_off_loans = filtered_df[filtered_df['loan_status'] == 'Charged Off']

# Calculate the percentage of charged off loans
charged_off_percentage = (charged_off_loans.shape[0] / filtered_df.shape[0]) * 100

# Calculate the total amount paid towards charged off loans
total_payment_charged_off = charged_off_loans['total_payment'].sum()

In [ ]:
print(f"Percentage of charged off loans: {charged_off_percentage:.2f}%")
print(f"Total amount paid towards charged off loans: {total_payment_charged_off:.2f}")

In [ ]:
charged_off_loans_loan_amount_sum = round(charged_off_loans['loan_amount'].sum(),2)
print(f' Total amount to be paid: {charged_off_loans_loan_amount_sum}')

charged_off_loans_total_payment_sum = round(charged_off_loans['total_payment'].sum(),2)
print(f' Total that has been paid: {charged_off_loans_total_payment_sum}')

charged_off_loans_loss = round(charged_off_loans_loan_amount_sum - charged_off_loans_total_payment_sum, 2)
print(f' Total that has been lost: {charged_off_loans_loss}')

In [ ]:
data = {'Paid': [charged_off_loans_total_payment_sum],
        'Loss': [charged_off_loans_loss]}

df = pd.DataFrame(data)
display(df)

In [ ]:
# Creating a pie chart
labels = ['Paid', 'Loss']
sizes = [charged_off_loans_total_payment_sum, charged_off_loans_loss]
colors = ['#1f77b4', '#ff7f0e']
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%')
# Setting the title of the pie chart
plt.title('Paid vs. Loss')
# Displaying the pie chart
plt.show()

In [ ]:
# Creating a pie chart
labels = ['Paid', 'Loss']
sizes = [charged_off_loans_total_payment_sum, charged_off_loans_loss]
colors = ['#1f77b4', '#ff7f0e']
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
# Setting the title of the pie chart
plt.title('Paid vs. Loss')
# Adding a legend
plt.legend()
# Displaying the pie chart
plt.show()

In [ ]:
filtered_df['num_of_payments_made'] = filtered_df['total_payment'] / filtered_df['instalment']
filtered_df['months_left_to_pay'] = filtered_df['term'] - filtered_df['num_of_payments_made']

In [ ]:
# Filtering the DataFrame to include only late loans
risk_loans = filtered_df[(filtered_df['loan_status'] == 'Late (31-120 days)') | (filtered_df['loan_status'] == 'Late (16-30 days)')]
# Calculating the percentage of risk loans
risk_percentage = round((risk_loans.shape[0] / filtered_df.shape[0]) * 100, 2)
# Calculating the total amount paid towards charged off loans
total_customers_in_risk_bracket = round(risk_loans.shape[0],0)
print(f"Percentage of risk loans: {risk_percentage}%")
print(f"Total number of customers in risk bracket: {total_customers_in_risk_bracket}")

In [ ]:
risk_loans_loan_amount_sum = round(risk_loans['loan_amount'].sum(),2)
# print(f' Total amount to be paid: {risk_loans_loan_amount_sum}')
risk_loans_total_payment_sum = round(risk_loans['total_payment'].sum(),2)
# print(f' Total that has been paid: {risk_loans_total_payment_sum}')
risk_loans_loss = round(risk_loans_loan_amount_sum - risk_loans_total_payment_sum, 2)
print(f'Total still to be paid (exclusive of int_rate): {risk_loans_loss}')
# print(f'Percentage: {round((risk_loans_loss/risk_loans_loan_amount_sum)*100,2)}%')

In [ ]:
risk_amount_left_to_pay_ = risk_loans['months_left_to_pay'] * risk_loans['instalment']
projected_loss = round(risk_amount_left_to_pay_.sum(), 2)
print(f"Projected Loss if Switched to Charged Off: ${projected_loss}")

In [ ]:
risk_charged_off_rev_pct = risk_percentage + charged_off_percentage 
print(f'Percentage of late and charged off revenue: {risk_charged_off_rev_pct:.2f}%')

In [ ]:
# compare grade, purpose and home_ownership with 
# customers who have already stopped paying 
# and 
# customers who are currently behind on payments.

# im also going to investigate employment_length, sub_grade, annual_inc and open_accounts

# Does the grade of the loan have effect on customers not paying?
# Is the purpose for the loan likely to have an effect?
# Does the home_ownership value contribute to the likelihood a customer won't pay?


In [ ]:
paid_loans = filtered_df[(filtered_df['loan_status'] == 'Fully Paid')]
on_time_loans = filtered_df[(filtered_df['loan_status'] == 'Current')]
grace_period_loans = filtered_df[(filtered_df['loan_status'] == 'In Grace Period')]

In [ ]:
data = {'Paid Grades': paid_loans['grade'].value_counts(),
        # 'On Time Grades': on_time_loans['grade'].value_counts(),
        # 'Grace Period Grades': grace_period_loans['grade'].value_counts(),
        'Risk Loan Grades': risk_loans['grade'].value_counts(),
        'Charged Off Grades': charged_off_loans['grade'].value_counts()
        }

df = pd.DataFrame(data)

for index, row in df.iterrows():
    print(f"Paid Loans {index} Grade: {row['Paid Grades']}")
    # print(f"On time {index} Grades: {row['On Time Grades']}")
    # print(f"Grace Period {index} Grades: {row['Grace Period Grades']}")
    print(f"Risk Loan {index} Grades: {row['Risk Loan Grades']}")
    print(f"Charged Off Loan {index} Grades: {row['Charged Off Grades']}") 

df.plot(kind='bar', stacked=True)

plt.title('Counts of Grades')
plt.xlabel('Grade')
plt.ylabel('Count')
plt.legend()
# Displaying the plot
plt.show()

In [ ]:
data = {'Paid sub grade': paid_loans['sub_grade'].value_counts(),
        # 'On Time sub grade': on_time_loans['sub_grade'].value_counts(),
        # 'Grace Period sub grade': grace_period_loans['sub_grade'].value_counts(),
        'Risk Loan sub grade': risk_loans['sub_grade'].value_counts(),
        'Charged Off sub grade': charged_off_loans['sub_grade'].value_counts()
        }

df = pd.DataFrame(data)

for index, row in df.iterrows():
    print(f"Paid Loans sub grade {index}: {row['Paid sub grade']}")
    # print(f"On time sub grade {index}: {row['On Time sub grade']}")
    # print(f"Grace Period sub grade {index}: {row['Grace Period sub grade']}")
    print(f"Risk Loan sub grade {index}: {row['Risk Loan sub grade']}")
    print(f"Charged Off Loan sub grade {index}: {row['Charged Off sub grade']}")

df = pd.DataFrame(data)

df.plot(kind='bar', stacked=True)

plt.title('Count of Sub Grade')
plt.xlabel('Sub Grades')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# next plot
data = {'Paid purpose': paid_loans['purpose'].value_counts(),
        # 'On Time purpose': on_time_loans['purpose'].value_counts(),
        # 'Grace Period purpose': grace_period_loans['purpose'].value_counts(),
        'Risk Loan purpose': risk_loans['purpose'].value_counts(),
        'Charged Off purpose': charged_off_loans['purpose'].value_counts()
        }

df = pd.DataFrame(data)

for index, row in df.iterrows():
    print(f"Paid Loans purpose: {index} {row['Paid purpose']}")
    # print(f"On time purpose: {index} {row['On Time purpose']}")
    # print(f"Grace Period purpose: {index} {row['Grace Period purpose']}")
    print(f"Risk Loan purpose: {index} {row['Risk Loan purpose']}")
    print(f"Charged Off Loan purpose: {index} {row['Charged Off purpose']}")

df.plot(kind='bar', stacked=True)

plt.title('Counts of Purposes')
plt.xlabel('Loan Purpose')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# next plot
data = {'Paid home ownership': paid_loans['home_ownership'].value_counts(),
    # 'On Time home ownership': on_time_loans['home_ownership'].value_counts(),
    # 'Grace Period home ownership': grace_period_loans['home_ownership'].value_counts(),
    'Risk Loan home ownership': risk_loans['home_ownership'].value_counts(),
    'Charged Off home ownership': charged_off_loans['home_ownership'].value_counts()
    }
df = pd.DataFrame(data)

for index, row in df.iterrows():
    print(f"Paid Loans home ownership: {index} {row['Paid home ownership']}")
    # print(f"On time home ownership: {index} {row['On Time home ownership']}")
    # print(f"Grace Period home ownership: {index} {row['Grace Period home ownership']}")
    print(f"Risk Loan home ownership: {index} {row['Risk Loan home ownership']}")
    print(f"Charged Off Loan home ownership: {index} {row['Charged Off home ownership']}")

df.plot(kind='bar', stacked=True)

plt.title('Counts of Home Ownership Status')
plt.xlabel('Home Ownership')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# next plot 
data = {'Paid employment length': paid_loans['employment_length'].value_counts(),
    # 'On Time employment length': on_time_loans['employment_length'].value_counts(),
    # 'Grace Period employment length': grace_period_loans['employment_length'].value_counts(),
    'Risk Loan employment length': risk_loans['employment_length'].value_counts(),
    'Charged Off employment length': charged_off_loans['employment_length'].value_counts()
    }

df = pd.DataFrame(data)

for index, row in df.iterrows():
    print(f"Paid Loans employment length {index}: {row['Paid employment length']}")
    # print(f"On time employment length {index}: {row['On Time employment length']}")
    # print(f"Grace Period employment length {index}: {row['Grace Period employment length']}")
    print(f"Risk Loan employment length {index}: {row['Risk Loan employment length']}")
    print(f"Charged Off Loan employment length {index}: {row['Charged Off employment length']}")

df.plot(kind='bar', stacked=True)

plt.title('Length of Employment')
plt.xlabel('Employment Length')
plt.ylabel('Count')
plt.legend()
plt.show()

In [ ]:
# next plot
data = {
'Paid annual income': paid_loans['annual_inc'],
'Risk Loan annual income': risk_loans['annual_inc'],
'Charged Off annual income': charged_off_loans['annual_inc']
}

df = pd.DataFrame(data)

# Print the counts for each loan category
for column in df.columns:
    counts = df[column].value_counts()
    print(f"{column} counts:")
    print(counts)
    print()

# Plotting histogram-density plots using seaborn
for column in df.columns:
    plt.figure(figsize=(10, 6))  # Set the figure size as needed
    plt.subplot(1, 2, 1)
    sns.histplot(df[column], kde=True)
    plt.title(f"Histogram-Density Plot - {column}")
    plt.xlabel("Annual Income")
    plt.ylabel("Density")
    plt.xlim(0, 700000)  # Adjust the x-axis limits as needed

    plt.subplot(1, 2, 2)
    sns.boxplot(y=df[column])
    plt.title(f"Box Plot - {column}")
    plt.ylabel("Annual Income")

    # Customising y-axis tick labels
    plt.gca().get_yaxis().get_major_formatter().set_scientific(False)
    plt.tight_layout()  # Adjust the spacing between subplots
    plt.show()

In [ ]:
# different view of box plots for annual_inc

data = {'Paid annual income': paid_loans['annual_inc'].value_counts(),
        # 'On Time annual income': on_time_loans['annual_inc'].value_counts(),
        # 'Grace Period annual income': grace_period_loans['annual_inc'].value_counts(),
        'Risk Loan annual income': risk_loans['annual_inc'].value_counts(),
        'Charged Off annual income': charged_off_loans['annual_inc'].value_counts()
        }

df = pd.DataFrame(data)

for index, row in df.iterrows():
    print(f"Paid Loans annual income {index}: {row['Paid annual income']}")
    # print(f"On time annual income {index}: {row['On Time annual income']}")
    # print(f"Grace Period annual income {index}: {row['Grace Period annual income']}")
    print(f"Risk Loan annual income {index}: {row['Risk Loan annual income']}")
    print(f"Charged Off Loan annual income {index}: {row['Charged Off annual income']}")

df = pd.DataFrame(data)

# Plotting the box plot
df.boxplot()
plt.title('Distribution of Annual Income')
plt.ylabel('Annual Income')
plt.xticks(rotation=45)

In [ ]:
# another view
data = {
    'Paid annual income': paid_loans['annual_inc'],
    'Risk Loan annual income': risk_loans['annual_inc'],
    'Charged Off annual income': charged_off_loans['annual_inc']
}

df = pd.DataFrame(data)

# Print the counts for each loan category
for column in df.columns:
    counts = df[column].value_counts()
    print(f"{column} counts:")
    print(counts)
    print()

df.boxplot()

# Customizing the plot
plt.title('Distribution of Annual Income')
plt.ylabel('Annual Income')
plt.xticks(rotation=45)

plt.show()

In [ ]:
# %%
# Summary:

# Grade / Sub Grade:
# Those with a higher Grade or Sub grade were more likely to pay back the full loan, than those with lower grades
# As the grade decends from 'A', there is a decrease in fully paid loans and an increase in risk and charged off loans between 'B1-E2'.
# Thus, the lower the grade the more risky the loan becomes.

# Annual Income:
# in comparison to the paid loans, charged off and risk loans are drastically smaller
# Thus indicating that a higher income is suggestive of being able to pay the full loan back
# And smaller annual incomes are suggestive of needing more time/assistance or likely to be charged off 

# Purpose:
# debt_consolidation Has a significant increase within the risk and charged off loans as a the primary/majority purpose for their loans
# The majority of paid loans was for this purpose too
# Meaning purpose alone may not directly be linked to likelyhood to be able to pay back the loans, an analysis of all indicators would need to be observed
# Other than that small businesses and credit_card purposes also poses a strong likelyhood of being charged off

# Home Ownership:
# Morgage owners are very likly to be able to pay the full loan
# Renters are slightly more likely to be charged off, but are still hold a good place for repaying their loans in full
# however, people who own their property seem to be not as likely as the previous two to pay the loan in full
# but there is less data on that point
# so in summary people who rent their property should be accepted with some caution 

# Employment Length:
# The largest amount of paid, charged off and risk loans all come frome people who have been in employment for 10 years+
# But the paid loans are significantly higher, suggesting that people in long term employment are more likely to pay their loans in full.
# However, ignoring the 10years+ most of the columns seem largely similar
# This could suggest that employment length doesnt have a significant impact on ability to repay loans
# but it should still be noted that people who have worked less than a year have the 2nd largest amount of charged off loans after the 10years+
